#Library and functions

In [ ]:
!pip install scikit-learn
!pip install fancyimpute
!pip install DistributedMissForest
!pip install MissForest

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 6.5 MB/s eta 0:00:00
  Created wheel for fancyimpute: filename=fancyimpute-0.7.0-py3-none-any.whl size=29879 sha256=81e17a5c5cc359e45dd88ae5cef5ce11e1f85daae6fc6dad1c67630a03a2246d
  Stored in directory: /root/.cache/pip/wheels/7b/0c/d3/ee82d1fbdcc0858d96434af108608d01703505d453720c84ed
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11329 sha256=35b1c5ddfa665580c6a9d57fd7e2092b5d475ba3c159aba6e5f3399f4a6b443d
  Stored in directory: /root/.cache/pip/wheels/46/06/a5/45a724630562413c374e29c08732411d496092408b3a7bf754
Successfully built fancyimpute knnimpute
  Preparing metadata (setup.py) ... done
  Created wheel for DistributedMissForest: filename=DistributedMissForest-1.4-py3-none-any.whl size=17469 sha256=a76c8f4acefcced33061609937647cf6e877752d40cf9a988fd1d1fe976002ec
  Stored in directory: /root/.cache/pip/wheels

In [ ]:
import numpy as np
import pandas as pd
import time
import math

import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

from sklearn import datasets
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as skLDA
from sklearn.experimental import enable_iterative_imputer
from sklearn.model_selection import train_test_split
from sklearn.impute import IterativeImputer

from scipy import stats
from fancyimpute import SoftImpute
from missforest.missforest import MissForest
from numpy.linalg import norm, inv

In [ ]:
#finding root closest CD
def solving(a,b,c,d,del_case):
  roots = np.roots([a,b,c,d])
  real_roots = np.real(roots[np.isreal(roots)])
  if len(real_roots)==1:
    return real_roots[0]
  else:
    f = lambda x: abs(x-del_case)
    F=[f(x) for x in real_roots]
    return real_roots[np.argmin(F)]

#variance matrix err
def error(sig, sig_est):
  er = np.linalg.norm(sig_est.flatten()-sig.flatten())/sig.size
  return er

#normalizing data
def normalize_data(X):
  scaler = StandardScaler()
  scaler.fit(X)
  return scaler.transform(X)

#generating NaN
def generate_nan(X, missing_rate):
  na_id = np.random.randint(0,X.size,round(missing_rate*X.size))
  Xnan = X.flatten()
  Xnan[na_id] = np.nan
  return Xnan.reshape(X.shape)

#Algorithms

In [ ]:
#For single class
def sig_estimate(X,mus0,mus1):
  m=n=l=sig11=sig22=s11=s12=s22=0
  del_case=0
  for i in X.T:
    if np.isfinite(i[0]) and np.isfinite(i[1]):
      m=m+1
      s11=s11+(i[0]-mus0)**2
      s22=s22+(i[1]-mus1)**2
      s12=s12+(i[0]-mus0)*(i[1]-mus1)
      sig11=sig11+(i[0]-mus0)**2
      sig22=sig22+(i[1]-mus1)**2
      del_case=del_case+(i[0]-mus0)*(i[1]-mus1)
    elif np.isfinite(i[0]) and np.isnan(i[1]):
      n=n+1
      sig11=sig11+(i[0]-mus0)**2
    elif np.isnan(i[0]) and np.isfinite(i[1]):
      l=l+1
      sig22=sig22+(i[1]-mus1)**2
  del_case = max(del_case/(m-1),0)
  sig11=sig11/(m+n)
  sig22=sig22/(m+l)
  sig12=solving(-m,s12,(m*sig11*sig22-s22*sig11-s11*sig22),s12*sig11*sig22,del_case)
  return sig11,sig22,sig12

def DPER(X):
  sig=np.zeros((X.shape[1],X.shape[1]))     #estimated covariance matrix
  #estimation of mean
  mu=np.nanmean(X,axis=0)
  #estimation of covariane
  for a in range(X.shape[1]):
    for b in range(a):
      temp=sig_estimate(np.array([X[:,b],X[:,a]]),mu[b],mu[a])
      sig[b][b]=temp[0]
      sig[a][a]=temp[1]
      sig[b][a]=sig[a][b]=temp[2]
  return sig

In [ ]:
#For multiclass (X,y) where y is a class
def sig_estimate_multi(X,mu0,mu1,y):
  del_case=0
  res=np.array([0]*8)  # [m,n,l,s11,s12,s22,sig11,sig22]
  numlabel=len(np.unique(y))
  for g in range(numlabel):
    m=n=l=s11=s12=s22=sig11=sig22=0
    mus0=mu0[g]
    mus1=mu1[g]
    Xg=(X.T)[y==g]
    for i in Xg:
      if np.isfinite(i[0]) and np.isfinite(i[1]):
        m=m+1
        s11=s11+(i[0]-mus0)**2
        s22=s22+(i[1]-mus1)**2
        s12=s12+(i[0]-mus0)*(i[1]-mus1)
        sig11=sig11+(i[0]-mus0)**2
        sig22=sig22+(i[1]-mus1)**2
      elif np.isfinite(i[0]) and np.isnan(i[1]):
        n=n+1
        sig11=sig11+(i[0]-mus0)**2
      elif np.isnan(i[0]) and np.isfinite(i[1]):
        l=l+1
        sig22=sig22+(i[1]-mus1)**2
    res = res+np.array([m,n,l,s11,s12,s22,sig11,sig22])
  m,n,l,s11,s12,s22,sig11,sig22 = res
  del_case = max(0,del_case/(m-1))
  sig11=sig11/(m+n)
  sig22=sig22/(m+l)
  sig12=solving(-m,s12,(m*sig11*sig22-s22*sig11-s11*sig22),s12*sig11*sig22,del_case)
  return sig11,sig22,sig12

def DPERmulticlass(X,y):            #with assumption of equal covariance matrices
  numlabel=len(np.unique(y))        #number of unique label in y
  p=X.shape[1]
  sig=np.zeros((p,p))               #estimated covariance matrix
  #compute mu_est
  mu=np.array([np.nanmean(X[y==g],axis=0) for g in range(numlabel)])
  #estimation of covariane matrix
  for a in range(p):
    for b in range(a):
      temp=sig_estimate_multi(np.array([X[:,b],X[:,a]]),mu[:,b],mu[:,a],y)
      sig[b][b]=temp[0]
      sig[a][a]=temp[1]
      sig[b][a]=sig[a][b]=temp[2]
  return sig

#Simulated dataset

In [ ]:
S0 = np.array([[1.,0.15,0.7,0.45],[0.15,1.,0.2,0.4],[0.7,0.2,1.,0.15],[0.45,0.4,0.15,1.]])
def simulation(run_time, missing_rate):
    G, n_per_class = 4, 100
    mu_1, mu_2 = np.array([1.5,2,1.7,3]), np.array([1,-2,1.5,-3])
    mu_3, mu_4 = np.array([1,3,1.7,1]), np.array([0,2,5,-1])
    err_single = []
    err_equal = []
    err_not = []
    for i in range(run_time):
        X_1 = np.random.multivariate_normal(mu_1,S0,n_per_class)
        X_2 = np.random.multivariate_normal(mu_2,S0,n_per_class)
        X_3 = np.random.multivariate_normal(mu_3,S0,n_per_class)
        X_4 = np.random.multivariate_normal(mu_4,S0,n_per_class)
        X = np.vstack((X_1,X_2, X_3, X_4))
        y = np.hstack((np.repeat(0,n_per_class), np.repeat(1,n_per_class),
                        np.repeat(2,n_per_class),np.repeat(3,n_per_class)))

        Xnan = generate_nan(X, missing_rate)

        #Impute data
        XMice = IterativeImputer(max_iter = 20).fit(Xnan).transform(Xnan)
        Xd = pd.DataFrame.from_records(Xnan)
        mf = MissForest()
        XMiss_df = mf.fit_transform(Xd)
        XMiss = XMiss_df.to_numpy()
        XSoft =  SoftImpute(max_iters = 20, verbose = False).fit_transform(Xnan)

        #Not assume equal covariance matrices
        S2 = np.array([S0 for g in range(G)])
        SDper2 = np.array([DPERmulticlass(Xnan,y) for g in range(G)])
        SMice2 = np.array([np.cov(XMice[y==g], rowvar = False) for g in range(G)])
        SMiss2 = np.array([np.cov(XMiss[y==g], rowvar = False) for g in range(G)])
        SSoft2 = np.array([np.cov(XSoft[y==g], rowvar = False) for g in range(G)])
        err_not.append(np.array([error(S2, SDper2),error(S2, SMice2), error(S2, SMiss2),error(S2, SSoft2)]))

        #Assume equal covariance matrices
        SDper = DPERmulticlass(Xnan,y)
        SMice =  sum([sum(y==g)*np.cov(XMice[y==g], rowvar = False) for g in range(G)])/len(y)
        SMiss =  sum([sum(y==g)*np.cov(XMiss[y==g], rowvar = False) for g in range(G)])/len(y)
        SSoft =  sum([sum(y==g)*np.cov(XSoft[y==g], rowvar = False) for g in range(G)])/len(y)
        err_equal.append(np.array([error(S0, SDper),error(S0, SMice), error(S0, SMiss),error(S0, SSoft)]))

        '''#Single class
        S_single = np.cov(X, rowvar = False)
        SDper_single = DPER(Xnan)
        SMiss_single = np.cov(XMiss, rowvar = False)
        SMice_single = np.cov(XMice, rowvar = False)
        SSoft_single = np.cov(XSoft, rowvar = False)
        err_single.append(np.array([error(SDper_single,S_single),error(SMiss_single,S_single),error(SMice_single,S_single),error(SSoft_single,S_single)]))'''
    return  err_not, err_equal

In [ ]:
run_time = 200
e20,ee20 = simulation(run_time,.2)
e40,ee40 = simulation(run_time,.4)
e60,ee60 = simulation(run_time,.6)
e80,ee80 = simulation(run_time,.8)

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Data

In [ ]:
pm = pd.DataFrame(np.repeat(" ± ",16).reshape(4,4),
                  index = ["20%", "40%", "60%", "80%"],
                  columns = ["DPER", "MICE", "MissForest", "Soft-Impute"])
print('Without the assumption of equal covariance matrices')
not_err = pd.DataFrame(np.vstack((np.mean(e20, axis = 0), np.mean(e40, axis = 0),
                      np.mean(e60, axis = 0),np.mean(e80, axis = 0))).round(3),
                      index = pm.index,
                      columns = pm.columns)
not_std = pd.DataFrame(np.vstack((np.std(e20, axis = 0), np.std(e40, axis = 0),
                      np.std(e60, axis = 0),np.std(e80, axis = 0))).round(3),
                      index = pm.index,
                      columns = pm.columns)
print(not_err.astype(str)+pm+not_std.astype(str))
print("\n")
print('Under the assumption of equal covariance matrices')
equal_err = pd.DataFrame(np.vstack((np.mean(ee20, axis = 0), np.mean(ee40, axis = 0),
                        np.mean(ee60, axis = 0),np.mean(ee80, axis = 0))).round(3),
                        index = ["20%", "40%", "60%", "80%"],
                        columns = ["DPER", "MICE", "MissForest", "Soft-Impute"])
equal_std = pd.DataFrame(np.vstack((np.std(ee20, axis = 0), np.std(ee40, axis = 0),
                      np.std(ee60, axis = 0),np.std(ee80, axis = 0))).round(3),
                      index = pm.index,
                      columns = pm.columns)
print(equal_err.astype(str)+pm+equal_std.astype(str))

Without the assumption of equal covariance matrices
              DPER           MICE     MissForest    Soft-Impute
20%  0.008 ± 0.003  0.027 ± 0.004  0.033 ± 0.008  0.059 ± 0.007
40%  0.009 ± 0.003  0.049 ± 0.006  0.069 ± 0.014  0.089 ± 0.006
60%  0.011 ± 0.003  0.068 ± 0.007  0.074 ± 0.012  0.102 ± 0.004
80%  0.013 ± 0.004  0.081 ± 0.008  0.073 ± 0.008  0.102 ± 0.006


Under the assumption of equal covariance matrices
              DPER           MICE     MissForest    Soft-Impute
20%  0.016 ± 0.005  0.039 ± 0.007  0.035 ± 0.009   0.07 ± 0.007
40%  0.018 ± 0.005   0.08 ± 0.009  0.087 ± 0.019  0.109 ± 0.007
60%  0.021 ± 0.006   0.11 ± 0.013  0.098 ± 0.022  0.123 ± 0.005
80%  0.025 ± 0.008  0.127 ± 0.014  0.094 ± 0.008  0.122 ± 0.007
